<a href="https://colab.research.google.com/github/JacopoMangiavacchi/FastLabeling/blob/master/mnist_odd_even.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai2>=0.0.11

In [2]:
from fastai2.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
path.ls()

(#4) [Path('oe_testing'),Path('testing'),Path('training'),Path('oe_training')]

In [4]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/2'),Path('training/5'),Path('training/3'),Path('training/8'),Path('training/1'),Path('training/4'),Path('training/7'),Path('training/6'),Path('training/9')]

In [5]:
(path/'oe_training').mkdir(exist_ok=True)
(path/'oe_training/odd').mkdir(exist_ok=True)
(path/'oe_training/even').mkdir(exist_ok=True)
(path/'oe_testing').mkdir(exist_ok=True)
(path/'oe_testing/odd').mkdir(exist_ok=True)
(path/'oe_testing/even').mkdir(exist_ok=True)
path.ls()

(#4) [Path('oe_testing'),Path('testing'),Path('training'),Path('oe_training')]

In [1]:
from shutil import copy2

def copy_files(path, source_folder, sub_folders_list, destination_folder):
  for i in sub_folders_list:
    folder = source_folder + '/' + str(i)
    for file in (path/folder).iterdir():
      copy2(file, (path/destination_folder))

In [ ]:
copy_files(path, 'training', [0, 2, 4, 6, 8], 'oe_training/odd')
copy_files(path, 'training', [1, 3, 5, 7, 9], 'oe_training/even')
copy_files(path, 'testing', [0, 2, 4, 6, 8], 'oe_testing/odd')
copy_files(path, 'testing', [1, 3, 5, 7, 9], 'oe_testing/even')


In [2]:
(path/'oe_training/odd').ls(), (path/'oe_training/even').ls(), (path/'oe_testing/odd').ls(), (path/'oe_testing/even').ls()

NameError: ignored

In [ ]:
dls = ImageDataLoaders.from_folder(path, train='training', valid='testing')

dls.train_ds, dls.valid_ds

((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(5)


epoch,train_loss,valid_loss,accuracy,time
0,0.609816,0.423266,0.868900,01:11


epoch,train_loss,valid_loss,accuracy,time
0,0.125911,0.068494,0.978900,01:18
1,0.074134,0.038765,0.988600,01:18
2,0.034743,0.024188,0.991800,01:17
3,0.014795,0.019734,0.994600,01:18
4,0.014940,0.019335,0.994400,01:18


In [ ]:
learn.validate()

In [ ]:
learn.summary()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)
interp.plot_top_losses(9, figsize=(7,7))

In [ ]:
test_path = (path/'testing'/'5').ls()[10]
test_image = Image.open(test_path)
test_image

In [ ]:
learn.predict(test_path)

('5',
 tensor(5),
 tensor([1.2365e-09, 4.4428e-10, 3.2736e-07, 2.2355e-07, 4.4440e-09, 1.0000e+00,
         1.1081e-07, 4.5415e-08, 4.7671e-09, 9.2808e-09]))